# Import Basic Libraries

In [19]:
import sklearn
import numpy as np
import pandas as pd

# Import Data

In [20]:
# training data
train = pd.read_csv("train.csv")
# test data
test = pd.read_csv("test_tweets.csv")

# Data Exploration (Exploratory Data Analysis)

In [21]:
train.head()

id  label                                              tweet
0   1      0   @user when a father is dysfunctional and is s...
1   2      0  @user @user thanks for #lyft credit i can't us...
2   3      0                                bihday your majesty
3   4      0  #model   i love u take with u all the time in ...
4   5      0             factsguide: society now    #motivation

In [59]:
test.head()

id                                              tweet  \
0  31963  #studiolife #aislife #requires #passion #dedic...   
1  31964   @user #white #supremacists want everyone to s...   
2  31965  safe ways to heal your #acne!!    #altwaystohe...   
3  31966  is the hp and the cursed child book up for res...   
4  31967    3rd #bihday to my amazing, hilarious #nephew...   

                                         clean_tweet  
0                                            to find  
1         want everyone to see the new and heres why  
2                           safe ways to heal your    
3  is the hp and the cursed child book up for res...  
4  rd to my amazing hilarious eli ahmir uncle dav...

In [23]:
#checking total count of non-racist/sexist related tweets
sum(train["label"] == 0)

29720

In [24]:
# checking total count of racist/sexist related tweets
sum(train["label"] == 1)

2242

In [25]:
# checking if there are any missing values
train.isnull().sum()
#train.isnull().values.any()

id       0
label    0
tweet    0
dtype: int64

# Data cleaning

In [26]:
#install tweet-preprocessor to clean tweets
!pip install tweet-preprocessor

In [27]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [28]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [29]:
# cleaning training data
train_tweet = clean_tweets(train["tweet"])
train_tweet = pd.DataFrame(train_tweet)

In [30]:
# appending cleaned tweets to the training data
train["clean_tweet"] = train_tweet
# comparing the cleaned and uncleaned tweets
train.head(10)

id  label                                              tweet  \
0   1      0   @user when a father is dysfunctional and is s...   
1   2      0  @user @user thanks for #lyft credit i can't us...   
2   3      0                                bihday your majesty   
3   4      0  #model   i love u take with u all the time in ...   
4   5      0             factsguide: society now    #motivation   
5   6      0  [2/2] huge fan fare and big talking before the...   
6   7      0   @user camping tomorrow @user @user @user @use...   
7   8      0  the next school year is the year for exams.ð...   
8   9      0  we won!!! love the land!!! #allin #cavs #champ...   
9  10      0   @user @user welcome here !  i'm   it's so #gr...   

                                         clean_tweet  
0  when a father is dysfunctional and is so selfi...  
1  thanks for credit i cant use cause they dont o...  
2                                bihday your majesty  
3            i love u take with u all the time in ur  
4                             factsguide society now  
5  2 2 huge fan fare and big talking before they ...  
6                             camping tomorrow danny  
7  the next school year is the year for exams can...  
8                               we won love the land  
9                           welcome here  im its so

In [31]:
# cleaning the test data and append the cleaned tweets to the test data
test_tweet = clean_tweets(test["tweet"])
test_tweet = pd.DataFrame(test_tweet)
# appending cleaned tweets to the test data
test["clean_tweet"] = test_tweet

# comparing the cleaned and uncleaned tweets
test.tail()

id                                              tweet  \
17192  49155  thought factory: left-right polarisation! #tru...   
17193  49156  feeling like a mermaid ð #hairflip #neverre...   
17194  49157  #hillary #campaigned today in #ohio((omg)) &am...   
17195  49158  happy, at work conference: right mindset leads...   
17196  49159  my   song "so glad" free download!  #shoegaze ...   

                                             clean_tweet  
17192       thought factory left right polarisation &gt3  
17193                             feeling like a mermaid  
17194  today in omg &amp used words like assets&ampli...  
17195  happy at work conference right mindset leads t...  
17196                      my song so glad free download

# Test and Train split


In [32]:
from sklearn.model_selection import train_test_split

# extracting the labels from the train data
y = train.label.values

# using 65% for the training and 35% for the test
x_train, x_test, y_train, y_test = train_test_split(train.clean_tweet.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.35, shuffle=True)

# Vectorize tweets using CountVectorizer

CountVectorizer Example

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
documents = ["This is Import Data's Youtube channel",
             "Data science is my passion and it is fun!",
             "Please subscribe to my channel"]

# initializing the countvectorizer
vectorizer = CountVectorizer()

# tokenizing and make the document into a matrix
document_term_matrix = vectorizer.fit_transform(documents)

# checking the result
pd.DataFrame(document_term_matrix.toarray(), columns = vectorizer.get_feature_names())

and  channel  data  fun  import  is  it  my  passion  please  science  \
0    0        1     1    0       1   1   0   0        0       0        0   
1    1        0     1    1       0   2   1   1        1       0        1   
2    0        1     0    0       0   0   0   1        0       1        0   

   subscribe  this  to  youtube  
0          0     1   0        1  
1          0     0   0        0  
2          1     0   1        0

In [81]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorizing tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learning a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(x_train) + list(x_test))

# transforming documents to document-term matrix
x_train_vec = vectorizer.transform(x_train)
x_test_vec = vectorizer.transform(x_test)

# Model building

Apply Support Vetor Classifier (SVC)

In [178]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel ='linear',gamma=0.01,C=100,probability=True)

# fitting the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# performing classification and prediction on samples in test data
y_pred_svm = svm.predict(x_test_vec)
#print(y_pred_svm)

 F1 score for SVC

In [184]:

from sklearn.metrics import f1_score

print("F1 score of prediction: ", f1_score(y_test, y_pred_svm  ) * 100, '%')



F1 score of prediction:  50.758459743290544 %


Importing predictions to file 

In [186]:
#Importing predictions to a csv file 
prediction=pd.DataFrame(y_pred_svm,columns=['Predictions']).to_csv('test_Prediction.csv')